In [1]:
import io
import json
from micromort.data_stores.mysql import db, cursor
from micromort.data_stores.mongodb import getConnection
from multiprocessing.dummy import Pool as ThreadPool
import json
from micromort.scrapers.newspaper_scraper import Newspaper_scraper
import traceback
import numpy as np
from bson import json_util
import pandas as pd
from datetime import timedelta

### Get tweets with lcoation:

In [1]:
cursor.execute('Select tweet_id, state, country from newsTweets_location where tweet_id > 1000232612659687424')
location_tweet_data = cursor.fetchall()

NameError: name 'cursor' is not defined

In [4]:
len(location_tweet_data)

1737081

### Fetch Urls from the tweets

In [211]:
mongo_db_name = "micromort"
mongo_collection_name = "newstweets_old"
mongoClient = getConnection(mongo_db_name, mongo_collection_name)
scraper = Newspaper_scraper(classify=False)

In [212]:
def getmongoData(tweet_id):
    return list(mongoClient.find({"id": tweet_id}))

def gen_dict_extract(key, var):
    if hasattr(var, 'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):
                for result in gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, list):
                for d in v:
                    for result in gen_dict_extract(key, d):
                        yield result

def getUrl(tweet):
    return gen_dict_extract("expanded_url", tweet)

In [7]:
counter=0
unique_urls ={}
def myFun(row):
    try:
        global counter
        global unique_urls
        counter=counter+1
        if counter%2000 == 0:
            print counter
        tweet_id = row[0]
        tweet = getmongoData(tweet_id)
        urls = getUrl(tweet[0])
        filtered_urls = {}
        for url in urls:
            url = url[:1000]
            if "twitter.com" not in url:
                filtered_urls[url] = 1
                unique_urls[url] = {}
        if not len(filtered_urls.keys()):
            return 1
        return {
            "urls" : filtered_urls.keys(),
            "tweet_id" : tweet_id,
            "state" : row[1],
            "country" : row[2]
        }
    except Exception as e:
        print e
        return -1
        
pool = ThreadPool(3)
location_url_tweets = pool.map(myFun, location_tweet_data)

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000
116000
118000
120000
122000
124000
126000
128000
130000
132000
134000
136000
138000
140000
142000
144000
146000
148000
150000
152000
154000
156000
158000
160000
162000
164000
166000
168000
170000
172000
174000
176000
178000
180000
182000
184000
186000
188000
190000
192000
194000
196000
198000
200000
202000
204000
206000
208000
210000
212000
214000
216000
218000
220000
222000
224000
226000
228000
230000
232000
234000
236000
238000
240000
242000
244000
246000
248000
250000
252000
254000
256000
258000
260000
262000
264000
266000
268000
270000
272000
274000
276000
278000
280000
282000
284000
286000
288000
290000
292000
294000
296000
298000
300000
302

### Scrape the data

In [8]:
#Number of unique urls:
len(unique_urls.keys())

179746

In [9]:
counter=0
def myFun(url):
    try:
        global counter
        counter=counter+1
        if counter%1000 == 0:
            print counter
        global unique_urls
        
        unique_urls[url] = scraper.scrape(url, False)
    except Exception as e:
        print e
    return 1

pool = ThreadPool(6)
a = pool.map(myFun, unique_urls.keys())

CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
/usr/local/lib/python2.7/dist-packages/dateutil/parser/_parser.py:1165: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this raise an exception.
  category=UnknownTimezoneWarning)
Building prefix dict from /home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/jieba/dict.txt ...
DEBUG:jieba:Building prefix dict from /home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/jieba/dict.txt ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.544285058975 seconds.
DEBUG:jieba:Loading model cost 0.544285058975 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.
CRITICAL:newspaper.article:jpeg error with PIL, timed out
CRITICAL:newspa

1000


CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
Traceback (most recent call last):
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml

2000


CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
Traceback (most recent call last):
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667

...

CRITICAL:newspaper.article:jpeg error with PIL, cannot concatenate 'str' and 'NoneType' objects
Traceback (most recent call last):
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml

178000


Traceback (most recent call last):
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582,

179000


Traceback (most recent call last):
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/newspaper/parsers.py", line 54, in fromstring
    cls.doc = lxml.html.fromstring(html)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 706, in fromstring
    doc = document_fromstring(html, parser=parser, base_url=base_url, **kw)
  File "/home/mannu/code/work/micromort/virtualenv/lib/python2.7/site-packages/lxml/html/__init__.py", line 600, in document_fromstring
    value = etree.fromstring(html, parser, **kw)
  File "lxml.etree.pyx", line 3032, in lxml.etree.fromstring (src/lxml/lxml.etree.c:68121)
  File "parser.pxi", line 1786, in lxml.etree._parseMemoryDocument (src/lxml/lxml.etree.c:102470)
  File "parser.pxi", line 1667, in lxml.etree._parseDoc (src/lxml/lxml.etree.c:101229)
  File "parser.pxi", line 1035, in lxml.etree._BaseParser._parseUnicodeDoc (src/lxml/lxml.etree.c:96139)
  File "parser.pxi", line 582,

In [17]:
len(unique_urls.keys())

179746

In [11]:
with open('unique_urls.json.final', 'w') as outfile:
    json.dump(unique_urls, outfile, default=json_util.default)

In [8]:
with open('unique_urls.json.final', 'r') as outfile:
    unique_urls = json.load(outfile, object_hook=json_util.object_hook)

### Get sentiment (dont forget to start the java serivce with multiple threads)

In [14]:
from micromort.models.polarity.stanford_deeply_moving import StanfordDeeplymoving
polarity_classifier = StanfordDeeplymoving()

In [15]:
counter=0
def myFun(url):
    try:
        global unique_urls
        global polarity_classifier
        global counter
        counter=counter+1
        if counter%1000 == 0:
            print counter
        d = unique_urls[url]["text"]
        polarity = polarity_classifier.get_sentiment_string(d.encode('ascii',errors='ignore')[:500])
        if polarity == "":
            polarity = "Neutral"
        unique_urls[url]["polarity"] = polarity
    except Exception as e:
        print e
    return 1
    
pool = ThreadPool(6) 
a = pool.map(myFun, unique_urls.keys())

'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
list index out of range
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' object has no attribute '__getitem__'
'int' ob

In [502]:
unique_urls

NameError: name 'unique_urls' is not defined

In [7]:
unique_urls[unique_urls.keys()[3]]["polarity"]

u'Positive'

### Get worry class

In [8]:
from micromort.models.trained_models.svm_mean_embeddings import Classifier, MeanEmbeddingVectorizer
classifier = Classifier("../resources/trained_models/oneVsAll_linear_SVM_mean_embeddings/")

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [503]:
count=0
def myFun(url):
    try:
        global count
        global unique_urls
        global classifier
        count=count+1
        d = unique_urls[url]["text"]
        labels = classifier.predict_single(d)
        if not len(labels):
            labels = [99]
        
        unique_urls[url]["labels"] = labels
        if count%1000 == 0:
            print labels, url
            print count

    except Exception as e:
        print e
        #traceback.print_exc()
        #print unique_urls[url] 
    return 1
    
pool = ThreadPool(6)
a = pool.map(myFun, unique_urls.keys())

SyntaxError: invalid syntax (<ipython-input-503-6c8afe5b846d>, line 1)

In [504]:
unique_urls[unique_urls.keys()[3]]

NameError: name 'aunique_urls' is not defined

### Merge location_url_tweet and unique_urls

In [9]:
count=0
def myFun(row):
    arr = []
    try:
        global unique_urls
        global count
        if row == 1 or row == -1:
            return arr
        count=count+1
        if count%20000 == 0:
            print count
        tweet_id = row["tweet_id"]
        urls = row["urls"]
        for url in urls:
            if unique_urls[url] == 1 or unique_urls[url] == -1:
                continue
            polarity = unique_urls[url]["polarity"]
            if polarity != "Negative" and polarity != 'Verynegative':
                continue
            worries = unique_urls[url]["labels"]
            for worry in worries:
                arr.append({
                    "tweet_id" : tweet_id,
                    "state" : row["state"],
                    "country" : row["country"],
                    "worry" : worry,
                    "polarity" : polarity,
                    "original_url" : unique_urls[url]["original_url"],
                    "url" : unique_urls[url]["url"]
                })
    except Exception as e:
        print e
        traceback.print_exc()
    return arr
pool = ThreadPool(6)
final_data = pool.map(myFun, location_url_tweets)

20000
 60000
80000
4000080000
100000
120000

140000
160000
180000
200000
 220000
240000
260000280000

300000
320000
320000
340000
360000360000

380000
400000
420000
440000
460000
480000
 480000
500000
500000
 520000
540000
560000
580000
600000
620000640000

660000660000
 
680000
700000
720000
740000
 740000
740000
760000
780000
800000
820000
840000
840000
860000
880000
880000
900000
920000
920000
940000
960000
980000
980000
1000000
10200001040000

1060000
1080000
11000001100000

 1100000
1120000
1140000
1160000
1180000
1200000
1220000
1240000
1260000
1280000


In [12]:
len(final_data)

1737081

In [10]:
x = [item[0] for item in final_data if len(item) !=0]

In [16]:
len(x)

1216270

In [17]:
final_data_df = pd.DataFrame(x)

In [21]:
final_data_df.head()

,country,original_url,polarity,state,tweet_id,url,worry
0,United States,https://mobile.nytimes.com/2018/04/26/us/polit...,Negative,Wisconsin,1000232615012683776,https://www.nytimes.com/2018/04/26/us/politics...,92
1,India,http://www.abplive.in/india-news/un-experts-ca...,Verynegative,,1000232615553785856,http://www.abplive.in/india-news/un-experts-ca...,99
2,United States,https://www.yahoo.com/news/trumps-son-concerne...,Negative,Oregon,1000232618732875776,https://www.yahoo.com/news/trumps-son-concerne...,92
3,United States,https://nyti.ms/2GPaLyJ,Negative,North Carolina,1000232619211149312,https://www.nytimes.com/2018/05/25/business/ec...,99
4,United States,https://www.yahoo.com/news/trumps-son-concerne...,Negative,Colorado,1000232619559215104,https://www.yahoo.com/news/trumps-son-concerne...,92


In [19]:
final_data_df.shape, final_data_df[final_data_df.worry <> 99].shape

((1216270, 7), (457988, 7))

In [25]:
final_data_df['original_url'] = final_data_df['original_url'].map(lambda x: x.encode('unicode-escape').decode('utf-8'))
#final_data_df.to_csv("./geo_worry_data.csv", header=True, index=False, encoding='utf-8')

In [33]:
import json
with open(name='../../data/final_results_set2.json', mode='w') as outfile:
    json.dump(unicode(x), outfile, ensure_ascii=False)

In [4]:
with io.open("../../data/final_results_set2.json", encoding='latin-1') as f:
    final_results = json.load(f)

In [5]:
import ast

In [6]:
json_data = ast.literal_eval(final_results)

In [7]:
json_data[:1]

[{'country': 'United States',
  'original_url': u'https://mobile.nytimes.com/2018/04/26/us/politics/migrant-children-missing.amp.html',
  'polarity': u'Negative',
  'state': 'Wisconsin',
  'tweet_id': 1000232615012683776L,
  'url': u'https://www.nytimes.com/2018/04/26/us/politics/migrant-children-missing.html',
  'worry': 92}]

In [8]:
final_results_df = pd.DataFrame(json_data)

In [9]:
final_results_df.head()

,country,original_url,polarity,state,tweet_id,url,worry
0,United States,https://mobile.nytimes.com/2018/04/26/us/polit...,Negative,Wisconsin,1000232615012683776,https://www.nytimes.com/2018/04/26/us/politics...,92
1,India,http://www.abplive.in/india-news/un-experts-ca...,Verynegative,,1000232615553785856,http://www.abplive.in/india-news/un-experts-ca...,99
2,United States,https://www.yahoo.com/news/trumps-son-concerne...,Negative,Oregon,1000232618732875776,https://www.yahoo.com/news/trumps-son-concerne...,92
3,United States,https://nyti.ms/2GPaLyJ,Negative,North Carolina,1000232619211149312,https://www.nytimes.com/2018/05/25/business/ec...,99
4,United States,https://www.yahoo.com/news/trumps-son-concerne...,Negative,Colorado,1000232619559215104,https://www.yahoo.com/news/trumps-son-concerne...,92


In [10]:
with io.open("../../data/final_results.json", encoding='latin-1') as f:
    final_results = json.load(f)

In [11]:
final_data_fil = []
for row in final_results:
    if row != 1:
        final_data_fil.append(row)

In [12]:
final_data_df_1 = pd.DataFrame(final_data_fil)

In [31]:
final_data_df_1[final_data_df_1.worry != 99].drop_duplicates()

(501106, 7)

In [29]:
final_results_df[final_results_df.worry != 99].shape

(457988, 7)

In [13]:
df = final_results_df[final_results_df.worry != 99].append(final_data_df_1[final_data_df_1.worry != 99])

In [14]:
df[df.worry != 99].shape, df.shape

((959094, 8), (959094, 8))

In [46]:
classes = {91: 'health', 92: 'safety_security', 93 : 'environment',
                      94 : 'social_relations', 95 : 'meaning_in_life', 96 : 'achievement',
                      97 : 'economics', 98 : 'politics', 99 : 'not_applicable', 0 : 'skip' }

In [47]:
worry_distribution = []
for row in pd.DataFrame({'count' : df.groupby( ["worry"] ).size()}).reset_index().values:
    worry_distribution.append([classes[row[0]], row[1]])

In [48]:
worry_distribution

[['health', 83661],
 ['safety_security', 337720],
 ['environment', 26860],
 ['social_relations', 165157],
 ['meaning_in_life', 17269],
 ['achievement', 172],
 ['economics', 19774],
 ['politics', 308481],
 ['not_applicable', 1599806]]

In [51]:
countries_above_the = df[df.worry <> 99].groupby( ["country"] ).size().reset_index()
countries_above_the_arr = []
for row in countries_above_the[countries_above_the[0] > 1000].values:
    countries_above_the_arr.append(row[0])
countries_above_the_arr

['Argentina',
 'Australia',
 'Bangladesh',
 'Belgium',
 'Brazil',
 'Canada',
 'Chile',
 'Colombia',
 'Finland',
 'France',
 'Germany',
 'India',
 'Indonesia',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Kenya',
 'Malaysia',
 'Mexico',
 'Netherlands',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Pakistan',
 'Philippines',
 'Qatar',
 'Singapore',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'Thailand',
 'The Netherlands',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 'Venezuela']

In [63]:
for row in pd.DataFrame({'count' : df[df.worry == 93].groupby( ["country"] ).size()}).reset_index().values:
    if row[0] not in countries_above_the_arr:
        continue
    normailzed = float(row[1] * 100)/ df[(df.country == row[0]) & (df.worry != 99)].shape[0]
    print row[0] + "," + str(normailzed)

Argentina,38.3812010444
Australia,7.95973263776
Bangladesh,2.2120518688
Belgium,4.11910669975
Brazil,5.89777195282
Canada,3.25150069263
Chile,34.7212420607
Colombia,77.5105231509
Finland,6.91126279863
France,7.36857434105
Germany,3.59792284866
India,1.1069384732
Indonesia,1.92743764172
Ireland,3.09054762335
Israel,2.06611570248
Italy,3.0637254902
Japan,1.29544504806
Kenya,5.38876058507
Malaysia,0.666615585013
Mexico,37.3524884556
Netherlands,6.12076095947
New Zealand,6.49350649351
Nigeria,3.06834030683
Norway,2.11992731678
Pakistan,0.899589892843
Philippines,2.77777777778
Qatar,2.7397260274
Singapore,2.66337854501
South Africa,2.84403669725
Spain,14.6723881829
Sweden,4.7668936616
Switzerland,6.5283709579
Thailand,4.70715055695
The Netherlands,3.6866359447
United Arab Emirates,1.39939485628
United Kingdom,2.20406186085
United States,1.37728807535
Venezuela,51.7775752051


In [69]:
_df = df.drop(["url", "original_url", "parsed_url"],axis=1)

In [71]:
_df.drop_duplicates().shape

/usr/local/lib/python2.7/dist-packages/pandas/core/algorithms.py:471: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  labels = table.get_labels(values, uniques, 0, na_sentinel, check_nulls)


(2558900, 5)

In [72]:
df.shape

(2558900, 8)

In [74]:
df_nw = df[df.worry != 99]

In [32]:
from bson.json_util import loads

### Get created at date for the tweets

In [15]:
df.head()

,country,original_url,parsed_url,polarity,state,tweet_id,url,worry
0,United States,https://mobile.nytimes.com/2018/04/26/us/polit...,NaN,Negative,Wisconsin,1000232615012683776,https://www.nytimes.com/2018/04/26/us/politics...,92
2,United States,https://www.yahoo.com/news/trumps-son-concerne...,NaN,Negative,Oregon,1000232618732875776,https://www.yahoo.com/news/trumps-son-concerne...,92
4,United States,https://www.yahoo.com/news/trumps-son-concerne...,NaN,Negative,Colorado,1000232619559215104,https://www.yahoo.com/news/trumps-son-concerne...,92
5,India,https://timesofindia.indiatimes.com/india/mega...,NaN,Negative,Madhya Pradesh,1000232621161431040,https://timesofindia.indiatimes.com/india/mega...,98
6,India,https://www.ndtv.com/video/news/left-right-cen...,NaN,Negative,,1000232625951289344,https://www.ndtv.com/video/news/left-right-cen...,92


In [20]:
counter=0

def myFun(row):
    try:
        global counter
        if counter%2000 == 0:
            print counter
        counter=counter+1
        row = row[1]
        tweet_id = row["tweet_id"]
        tweet = getmongoData(tweet_id)[0]
        return {
            "url" : row["url"],
            "tweet_id" : tweet_id,
            "country" : row["country"],
            "polarity" : row["polarity"],
            "state" : row["state"],
            "worry" : row["worry"],
            "created_at" : tweet["created_at"]
        }
    except Exception as e:
        print e
        return 1
        
pool = ThreadPool(6)
date_tweets = pool.imap(myFun, df.iterrows())

0
20002000

40004000

60006000

8000
1000010000

12000
 12000
14000
16000
18000
20000
2200022000

24000
2600026000

28000
30000
32000
34000
3600036000

 36000
38000
40000
42000
44000
 44000
46000
48000
50000
5200052000

54000
56000
58000
60000
6200062000

64000
6600066000

68000
70000
7200072000

74000
7600076000

78000
80000
82000
84000
 84000
8600086000

88000
90000
92000
9400094000

96000
98000
100000100000

102000
104000
106000
108000
110000
112000112000

114000
116000
118000
120000
 120000
122000
124000124000

126000
128000
130000
132000
134000
 134000
136000
138000
 138000
140000
142000142000

144000 
144000
146000
148000
 148000
148000
150000
152000152000

154000
156000
158000
160000
 160000160000

162000
164000
166000166000

168000168000

170000
172000
174000
176000
178000
180000180000

182000
184000
186000186000

188000
190000190000

192000
194000
196000196000196000


198000
200000200000

202000
204000
206000
208000
210000
212000
214000214000

216000
 216000
218000
220000
2220

In [21]:
a = list(date_tweets)

In [23]:
date_pd = pd.DataFrame(a)

In [2]:
df = pd.read_csv("../../data/all_with_dates.csv", encoding='latin-1')

In [8]:
df.head()

,Unnamed: 0,country,created_at,polarity,state,tweet_id,url,worry
0,0,United States,2018-05-26 04:30:13,Negative,Wisconsin,1000232615012683776,https://www.nytimes.com/2018/04/26/us/politics...,92
1,1,United States,2018-05-26 04:30:14,Negative,Oregon,1000232618732875776,https://www.yahoo.com/news/trumps-son-concerne...,92
2,2,United States,2018-05-26 04:30:14,Negative,Colorado,1000232619559215104,https://www.yahoo.com/news/trumps-son-concerne...,92
3,3,India,2018-05-26 04:30:14,Negative,Madhya Pradesh,1000232621161431040,https://timesofindia.indiatimes.com/india/mega...,98
4,4,India,2018-05-26 04:30:16,Negative,NaN,1000232625951289344,https://www.ndtv.com/video/news/left-right-cen...,92


In [6]:
for index, row in df.iterrows():
    df["created_at"].values[index] = pd.to_datetime(row.created_at)

In [9]:
df.head()

,Unnamed: 0,country,created_at,polarity,state,tweet_id,url,worry
0,0,United States,2018-05-26 04:30:13,Negative,Wisconsin,1000232615012683776,https://www.nytimes.com/2018/04/26/us/politics...,92
1,1,United States,2018-05-26 04:30:14,Negative,Oregon,1000232618732875776,https://www.yahoo.com/news/trumps-son-concerne...,92
2,2,United States,2018-05-26 04:30:14,Negative,Colorado,1000232619559215104,https://www.yahoo.com/news/trumps-son-concerne...,92
3,3,India,2018-05-26 04:30:14,Negative,Madhya Pradesh,1000232621161431040,https://timesofindia.indiatimes.com/india/mega...,98
4,4,India,2018-05-26 04:30:16,Negative,NaN,1000232625951289344,https://www.ndtv.com/video/news/left-right-cen...,92


In [10]:
US_states = ["Alabama" ,"Alaska" ,"Arizona" ,"Arkansas" ,"California" ,"Colorado" ,"Connecticut" ,"Delaware" ,"Florida" ,"Georgia" ,"Hawaii" ,"Idaho" ,"Illinois Indiana" ,"Iowa" ,"Kansas" ,"Kentucky" ,"Louisiana" ,"Maine" ,"Maryland" ,"Massachusetts" ,"Michigan" ,"Minnesota" ,"Mississippi" ,"Missouri" ,"Montana Nebraska" ,"Nevada" ,"New Hampshire" ,"New Jersey" ,"New Mexico" ,"New York" ,"North Carolina" ,"North Dakota" ,"Ohio" ,"Oklahoma" ,"Oregon" ,"Pennsylvania Rhode Island" ,"South Carolina" ,"South Dakota" ,"Tennessee" ,"Texas" ,"Utah" ,"Vermont" ,"Virginia" ,"Washington" ,"West Virginia" ,"Wisconsin" ,"Wyoming"]

In [12]:
df.to_csv("../../data/all_with_dates-formatted.csv", encoding='latin-1')

## Analysis

In [4]:
country_grp = pd.DataFrame({'counts' : df.groupby( ["country"] ).size()}).reset_index()

In [40]:
country_grp[country_grp.counts > 1000].shape

(38, 2)

In [63]:
df.created_at.min(), df.created_at.max()

(Timestamp('2018-05-14 09:18:23'), Timestamp('2018-06-05 03:09:38'))

### Week wise data

In [107]:
for i in range(0,4):
    period=7
    from_time = pd.to_datetime("2018-05-15 00:00:00") + timedelta(days=period*i)
    to_time = from_time + timedelta(days=period)
    print from_time, " to ", to_time, " : ", df[df.created_at > from_time ][ df.created_at < to_time].shape

2018-05-15 00:00:00  to  2018-05-22 00:00:00  : 

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


 (258643, 8)
2018-05-22 00:00:00  to  2018-05-29 00:00:00  :  (328209, 8)
2018-05-29 00:00:00  to  2018-06-05 00:00:00  :  (336516, 8)
2018-06-05 00:00:00  to  2018-06-12 00:00:00  :  (6826, 8)


In [120]:
from_time = pd.to_datetime("2018-05-17 00:00:00")
temp = df[df.created_at > from_time ][ df.created_at < from_time + timedelta(days=1)]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [121]:
temp = pd.DataFrame({"counts" : temp.groupby(["country"]).size()}).reset_index()

In [125]:
print temp[temp.counts > 800].shape
temp[temp.counts > 800]

(4, 2)


,country,counts
24,Canada,855
59,India,5944
137,United Kingdom,3386
138,United States,16627


In [142]:
countries = ["Canada", "India", "United Kingdom", "United States"]

In [140]:
df[df.created_at > from_time ][ df.created_at < to_time]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Unnamed: 0,country,created_at,polarity,state,tweet_id,url,worry


In [256]:
ob = {} 
for classes in range(91,100):
    ob[classes] = {
    "Canada" : [],
    "India" : [],
    "United Kingdom" : [],
    "United States" : []
}

In [257]:
ob

{91: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 92: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 93: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 94: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 95: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 96: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 97: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 98: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 99: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []}}

In [258]:
for i in range(0,21):
    period=1
    from_time = pd.to_datetime("2018-05-15 00:00:00") + timedelta(days=period*i)
    to_time = from_time + timedelta(days=period)
    
    _df = df[df.created_at > from_time ][ df.created_at < to_time]
    
    for country in countries:
        for worry in range(91,100):
            normailzed = float(_df[_df.country == country][_df.worry == worry].shape[0] * 100)/ _df[_df.country == country][_df.worry != 99].shape[0]
            ob[worry][country].append(normailzed)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [163]:
for country in countries:
    s = ', health,safety_security,environment, social_relations, meaning_in_life, achievement,economics, politics' + "\n"
    for i in range(0,21):
        s = s + ","
        for worry in range(91,99):
            s = s + str(ob[worry][country][i]) + ","
        s = s + "\n"
    print country + "\n" + s

Canada
, health,safety_security,environment, social_relations, meaning_in_life, achievement,economics, politics
,8.49909584087,23.869801085,3.34538878843,52.9837251356,1.17540687161,0.0,0.904159132007,9.22242314647,
,11.4125350795,15.0608044902,8.60617399439,32.3666978485,0.561272217025,0.187090739008,0.935453695042,30.8699719364,
,16.8421052632,23.0409356725,6.19883040936,27.134502924,1.98830409357,0.0,0.584795321637,24.2105263158,
,11.2299465241,41.7112299465,7.16577540107,17.7540106952,6.31016042781,0.0,1.28342245989,14.5454545455,
,7.59162303665,67.277486911,1.74520069808,8.6387434555,7.24258289703,0.0,0.087260034904,7.41710296684,
,11.5560640732,38.4439359268,2.5171624714,21.0526315789,6.17848970252,0.0,0.343249427918,19.9084668192,
,15.7509157509,20.5128205128,3.7851037851,18.8034188034,4.6398046398,0.0,0.3663003663,36.1416361416,
,17.3652694611,30.5389221557,3.29341317365,27.245508982,2.09580838323,0.0,0.59880239521,18.8622754491,
,17.4348697395,42.2845691383,5.61122244489,14.72

### Filter domain names

In [128]:
domain_arr = ["www.nytimes.com", "www.theguardian.com", "www.bbc.com", "www.yahoo.com", "www.ndtv.com", "mobile.nytimes.com", "timesofindia.indiatimes.com", "www.news18.com", "indianexpress.com", "www.indiatoday.in", "www.thehindu.com", "www.bbc.co.uk", "www.firstpost.com", "www.rappler.com", "edition.cnn.com", "amp.theguardian.com", "finance.yahoo.com", "www.thestar.com.my", "www.freemalaysiatoday.com"]

In [505]:
domain_list = [
    "www.nytimes.com", "mobile.nytimes.com", "thecaucus.blogs.nytimes.com", "cn.nytimes.com", "archive.nytimes.com", "learning.blogs.nytimes.com", "static.nytimes.com", "well.blogs.nytimes.com", "6thfloor.blogs.nytimes.com", "krugman.blogs.nytimes.com", "cityroom.blogs.nytimes.com", "cooking.nytimes.com", "dealbook.nytimes.com", "bits.blogs.nytimes.com", "opinionator.blogs.nytimes.com", "jobs.nytimes.com", "paidpost.nytimes.com", "timesevents.nytimes.com", "thelede.blogs.nytimes.com", "atwar.blogs.nytimes.com", "markets.on.nytimes.com", "fivethirtyeight.blogs.nytimes.com", "economix.blogs.nytimes.com", "kristof.blogs.nytimes.com", "newoldage.blogs.nytimes.com", "consults.blogs.nytimes.com", "iht-retrospective.blogs.nytimes.com", "therail.blogs.nytimes.com", "myaccount.nytimes.com", "rendezvous.blogs.nytimes.com",
    "www.theguardian.com", "amp.theguardian.com", "jobs.theguardian.com", "www.theguardian.com", "membership.theguardian.com", "witness.theguardian.com", "profile.theguardian.com", "www.theguardian.pe.ca",
    "www.bbc.com","www.bbc.co.uk","news.bbc.co.uk","bbcwatch.org",
    "www.yahoo.com", "finance.yahoo.com", "au.news.yahoo.com", "ca.news.yahoo.com", "in.news.yahoo.com", "uk.news.yahoo.com", "sg.news.yahoo.com", "in.finance.yahoo.com", "ca.finance.yahoo.com", "uk.finance.yahoo.com", "ph.news.yahoo.com", "au.finance.yahoo.com", "sports.yahoo.com", "sg.finance.yahoo.com", "sg.yahoo.com", "nz.news.yahoo.com", "ca.sports.yahoo.com", "uk.sports.yahoo.com", "nz.finance.yahoo.com", "view.yahoo.com", "sg.style.yahoo.com", "us.yahoo.com", "ca.style.yahoo.com", "in.style.yahoo.com", 
    "www.ndtv.com","gadgets.ndtv.com","swachhindia.ndtv.com","food.ndtv.com","auto.ndtv.com","sports.ndtv.com","special.ndtv.com","swirlster.ndtv.com","doctor.ndtv.com","drop.ndtv.com","movies.ndtv.com",
    "timesofindia.indiatimes.com", "blogs.timesofindia.indiatimes.com", "www.indiatimes.com", "economictimes.indiatimes.com", "bangaloremirror.indiatimes.com", "ahmedabadmirror.indiatimes.com", "punemirror.indiatimes.com", "ciso.economictimes.indiatimes.com", "energy.economictimes.indiatimes.com", "beautypageants.indiatimes.com", "cio.economictimes.indiatimes.com", 
    "www.news18.com",
    "indianexpress.com","archive.indianexpress.com","www.newindianexpress.com","expressindia.indianexpress.com","events.indianexpress.com","elections.indianexpress.com",
    "www.indiatoday.in","indiatoday.app.link",
    "www.thehindu.com","www.thehindubusinessline.com",
    "www.firstpost.com",
    "www.rappler.com","amp.rappler.com",
    "edition.cnn.com","edition-m.cnn.com","money.cnn.com","amp.cnn.com","cnnphilippines.com","www-m.cnn.com",
    "www.thestar.com.my","www.thestar.com","www.thestar.co.uk","m.thestar.com.my",
    "www.freemalaysiatoday.com",
    "www.channelnewsasia.com",
    "www.dnaindia.com",
    "www.asiaone.com",
    "japantoday.com",
    "www.philstar.com","youngstar.philstar.com",
    "www.malaymail.com","m.malaymail.com",
    "www.straitstimes.com","stomp.straitstimes.com",
    "www.japantimes.co.jp","bookclub.japantimes.co.jp",
    "www.tnp.sg", "www.theonlinecitizen.com", "www.thejakartapost.com", "jakartaglobe.id", "www.bangkokpost.com", "www.nationmultimedia.com", "www.gmanetwork.com", "www.interaksyon.com", "news.mb.com.ph", "business.mb.com.ph", "mb.com.ph", "lifestyle.mb.com.ph", "entertainment.mb.com.ph", "sports.mb.com.ph", "technology.mb.com.ph", "newsbits.mb.com.ph", "www.bernama.com", "thesundaily.my", "www.nst.com.my", "www1.nst.com.my", "www.malaysiandigest.com", "www.theedgemarkets.com", "www.asahi.com", "soranews24.com", "www.tokyoreporter.com", "mainichi.jp", "www.deccanchronicle.com", "deccanchronicle.com", "www.haaretz.com", "jakartaglobe.id", "says.com", "www.malaysiakini.com", "www.gmanetwork.com"
]

In [506]:
count=0
def myFun(row):
    try:
        global count
        count=count+1
        if count%2000 == 0:
            print count
        url = row[1]["url"]
        for domain in domain_list:
            if domain in url:
                return row
        return 1
    except Exception as e:
        print e
        return 1

pool = ThreadPool(6)
filtered_domains = pool.imap(myFun, df.iterrows())

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000
116000
118000
120000
122000
124000
126000
128000
130000
132000
134000
136000
138000
140000
142000
144000
146000
148000
150000
152000
154000
156000
158000
160000
162000
164000
166000
168000
170000
172000
174000
176000
178000
180000
182000
184000
186000
188000
190000
192000
194000
196000
198000
200000
202000
204000
206000
208000
210000
212000
214000
216000
218000
220000
222000
224000
226000
228000
230000
232000
234000
236000
238000
240000
242000
244000
246000
248000
250000
252000
254000
256000
258000
260000
262000
264000
266000
268000
270000
272000
274000
276000
278000
280000
282000
284000
286000
288000
290000
292000
294000
296000
298000
300000
302

In [507]:
filtered_domains = list(filtered_domains)

In [518]:
a = []
for r in filtered_domains:
    try:
        if r != 0 and r != 1:
            a.append(r[1])
    except Exception as e:
        print e, r
        #continue

In [519]:
new_df = pd.DataFrame(a)

In [520]:
new_df.shape, df.shape

((932927, 8), (959094, 8))

In [521]:
org = pd.DataFrame({'counts' : df.groupby( [ "country"] ).size()}).reset_index().sort_values("counts",ascending=False)

In [522]:
org2 = pd.DataFrame({'counts' : new_df.groupby( [ "country"] ).size()}).reset_index().sort_values("counts",ascending=False)

In [523]:
org2[org2.counts > 1000]

,country,counts
176,United States,470861
72,India,168861
175,United Kingdom,108966
30,Canada,25138
10,Australia,24109
97,Malaysia,12950
126,Philippines,9332
120,Pakistan,7351
81,Japan,7021
56,France,6483


In [524]:
org[org.counts > 1000]

,country,counts
176,United States,483922
72,India,172819
175,United Kingdom,111476
30,Canada,25988
10,Australia,24536
97,Malaysia,13051
126,Philippines,9504
120,Pakistan,7559
81,Japan,7179
56,France,6867


In [525]:
con = ["India","Malaysia","Philippines","Pakistan","Japan","Singapore","Indonesia","Thailand","Bangladesh","Israel"]

In [526]:
for c in con:
    print c, " % change in dec: ", 100 - float(org2[org2.country == c].counts*100)/org[org.country == c].counts

India  % change in dec:  72    2.290257
Name: counts, dtype: float64
Malaysia  % change in dec:  97    0.773887
Name: counts, dtype: float64
Philippines  % change in dec:  126    1.809764
Name: counts, dtype: float64
Pakistan  % change in dec:  120    2.751687
Name: counts, dtype: float64
Japan  % change in dec:  81    2.200864
Name: counts, dtype: float64
Singapore  % change in dec:  144    3.723798
Name: counts, dtype: float64
Indonesia  % change in dec:  73    4.563492
Name: counts, dtype: float64
Thailand  % change in dec:  163    10.27668
Name: counts, dtype: float64
Bangladesh  % change in dec:  14    5.034325
Name: counts, dtype: float64
Israel  % change in dec:  77    6.859504
Name: counts, dtype: float64


In [461]:
pd.DataFrame(a)

,Unnamed: 0,country,created_at,polarity,state,tweet_id,url,worry
0,0,United States,2018-05-26 04:30:13,Negative,Wisconsin,1000232615012683776,https://www.nytimes.com/2018/04/26/us/politics...,92
1,1,United States,2018-05-26 04:30:14,Negative,Oregon,1000232618732875776,https://www.yahoo.com/news/trumps-son-concerne...,92
2,2,United States,2018-05-26 04:30:14,Negative,Colorado,1000232619559215104,https://www.yahoo.com/news/trumps-son-concerne...,92
3,3,India,2018-05-26 04:30:14,Negative,Madhya Pradesh,1000232621161431040,https://timesofindia.indiatimes.com/india/mega...,98
4,4,India,2018-05-26 04:30:16,Negative,NaN,1000232625951289344,https://www.ndtv.com/video/news/left-right-cen...,92
5,5,India,2018-05-26 04:30:17,Negative,NaN,1000232632595054592,https://www.ndtv.com/video/news/left-right-cen...,92
6,6,Norway,2018-05-26 04:30:19,Negative,NaN,1000232642095136768,https://www.nytimes.com/2018/05/25/us/politics...,98
7,7,Canada,2018-05-26 04:30:21,Verynegative,Ontario,1000232646847311874,https://www.theguardian.com/us-news/2018/may/2...,92
8,8,India,2018-05-26 04:30:22,Negative,NaN,1000232653470056449,https://www.ndtv.com/video/news/left-right-cen...,92
9,9,United States,2018-05-26 04:30:23,Negative,NaN,1000232657039511552,https://www.yahoo.com/news/trumps-son-concerne...,92


In [444]:
filtered_domains = pd.DataFrame(f)

In [458]:
a = []
for r in f:
    a.append(r[1])
    break

In [193]:
_f = df[df.worry==97][df.country=="India"][df.created_at > pd.to_datetime("2018-05-30 00:00:00")][df.created_at < pd.to_datetime("2018-05-31 00:00:00")]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


### Get user IDs

In [209]:
df.head()

,Unnamed: 0,country,created_at,polarity,state,tweet_id,url,worry
0,0,United States,2018-05-26 04:30:13,Negative,Wisconsin,1000232615012683776,https://www.nytimes.com/2018/04/26/us/politics...,92
1,1,United States,2018-05-26 04:30:14,Negative,Oregon,1000232618732875776,https://www.yahoo.com/news/trumps-son-concerne...,92
2,2,United States,2018-05-26 04:30:14,Negative,Colorado,1000232619559215104,https://www.yahoo.com/news/trumps-son-concerne...,92
3,3,India,2018-05-26 04:30:14,Negative,Madhya Pradesh,1000232621161431040,https://timesofindia.indiatimes.com/india/mega...,98
4,4,India,2018-05-26 04:30:16,Negative,NaN,1000232625951289344,https://www.ndtv.com/video/news/left-right-cen...,92


In [215]:
counter=0

def myFun(row):
    try:
        global counter
        if counter%2000 == 0:
            print counter
        counter=counter+1
        row = row[1]
        tweet_id = row["tweet_id"]
        tweet = getmongoData(tweet_id)[0]
        return {
            "url" : row["url"],
            "tweet_id" : tweet_id,
            "country" : row["country"],
            "polarity" : row["polarity"],
            "state" : row["state"],
            "worry" : row["worry"],
            "created_at" : row["created_at"],
            "user_id" : tweet["user"]["id"]
        }
    except Exception as e:
        print e
        return 1
        
pool = ThreadPool(6)
user_id_tweets = pool.imap(myFun, df.iterrows())

00

2000
4000
60006000

8000
1000010000

12000
14000
16000
18000
20000
22000
2400024000

2600026000

2800028000

30000
3200032000

340003400034000


3600036000

3800038000

40000
42000
44000
46000
480004800048000


50000
5200052000

5400054000

5600056000

58000
60000
62000
64000
6600066000

68000
70000
7200072000

74000
76000
78000
80000
8200082000

8400084000

8600086000

88000
90000
92000
94000
96000
 96000
98000
100000
102000
 102000
104000104000

106000
108000108000

110000
112000
 112000
114000114000114000


116000116000

118000
120000
122000
124000
126000
128000128000

130000130000

132000
134000
136000
138000
140000
142000
144000
146000146000

148000
150000150000

152000
154000
156000156000

158000
 158000
160000160000

162000162000

164000
166000
168000168000

170000
172000172000172000


174000
176000176000

178000
180000
182000
184000184000

186000
188000
190000
192000
194000
196000196000
196000

198000
200000200000

202000202000

204000
206000206000

208000
210000
212000
214

In [216]:
a = list(user_id_tweets)

In [218]:
df2 = pd.DataFrame(a)

In [224]:
df2[df2.country == "India"][df2.worry == 97][df.created_at > pd.to_datetime("2018-05-30 00:00:00")][df.created_at < pd.to_datetime("2018-05-31 00:00:00")].shape

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(3963, 8)

In [227]:
len(df2[df2.country == "India"][df2.worry == 97][df.created_at > pd.to_datetime("2018-05-30 00:00:00")][df.created_at < pd.to_datetime("2018-05-31 00:00:00")].user_id.unique())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


3700

### Do the analysis user wise

In [234]:
ob = {} 
for classes in range(91,100):
    ob[classes] = {
    "Canada" : [],
    "India" : [],
    "United Kingdom" : [],
    "United States" : []
}

In [235]:
ob

{91: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 92: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 93: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 94: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 95: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 96: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 97: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 98: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []},
 99: {'Canada': [], 'India': [], 'United Kingdom': [], 'United States': []}}

In [ ]:
len(df2[df2.country == "India"][df2.worry == 97][df.created_at > pd.to_datetime("2018-05-30 00:00:00")][df.created_at < pd.to_datetime("2018-05-31 00:00:00")].user_id.unique())

In [236]:
for i in range(0,21):
    period=1
    from_time = pd.to_datetime("2018-05-15 00:00:00") + timedelta(days=period*i)
    to_time = from_time + timedelta(days=period)
    
    _df = df2[df2.created_at > from_time ][ df2.created_at < to_time]
    
    for country in countries:
        for worry in range(91,100):
            normailzed = float(len(_df[_df.country == country][_df.worry == worry].user_id.unique()) * 100)/ len(_df[_df.country == country][_df.worry != 99].user_id.unique())
            ob[worry][country].append(normailzed)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [237]:
for country in countries:
    s = ', health,safety_security,environment, social_relations, meaning_in_life, achievement,economics, politics' + "\n"
    for i in range(0,21):
        s = s + ","
        for worry in range(91,99):
            s = s + str(ob[worry][country][i]) + ","
        s = s + "\n"
    print country + "\n" + s

Canada
, health,safety_security,environment, social_relations, meaning_in_life, achievement,economics, politics
,11.0024449878,27.0171149144,4.1564792176,55.9902200489,1.58924205379,0.0,1.22249388753,10.1466992665,
,13.7086903305,17.2582619339,11.0159118727,36.3525091799,0.734394124847,0.244798041616,1.10159118727,29.6205630355,
,20.0301204819,25.7530120482,7.68072289157,28.313253012,2.40963855422,0.0,0.451807228916,25.6024096386,
,13.0376344086,45.564516129,8.19892473118,19.7580645161,7.79569892473,0.0,1.20967741935,13.7096774194,
,9.51302378256,68.9694224236,2.03850509626,9.62627406569,9.28652321631,0.0,0.113250283126,7.9275198188,
,14.0825035562,41.1095305832,2.56045519203,21.6216216216,7.6813655761,0.0,0.426742532006,21.7638691323,
,19.1290824261,22.2395023328,4.5101088647,21.7729393468,5.75427682737,0.0,0.466562986003,38.8802488336,
,21.0459183673,30.8673469388,4.08163265306,29.5918367347,2.67857142857,0.0,0.765306122449,20.6632653061,
,20.3346203346,43.8867438867,6.56370656371,16

### plots comaping countries

In [259]:
countries

['Canada', 'India', 'United Kingdom', 'United States']

In [254]:
for worry in range(91,99):
    s = ', Canada, India, UK, US' + "\n"
    for i in range(0,21):
        s = s + ","
        for country in countries:
            s = s + str(ob[worry][country][i]) + ","
        s = s + "\n"
    print str(worry) + "\n" + s

91
, India, Canada, UK, US
,8.49909584087,13.255121695,21.2900414034,7.47898020071,
,11.4125350795,4.33717579251,31.0091743119,5.29956210191,
,16.8421052632,2.27119784657,17.9267572357,8.75082696818,
,11.2299465241,3.02102185716,12.8211089155,9.38324007975,
,7.59162303665,12.4730826569,10.8193041526,6.40352522033,
,11.5560640732,3.54619384082,52.1548930655,6.32318501171,
,15.7509157509,5.48751200768,36.5955359333,7.88066908088,
,17.3652694611,7.03441295547,23.4462764994,11.3790266439,
,17.4348697395,7.31605351171,37.3788030759,11.9065514218,
,11.2261146497,10.2613538989,24.2868621937,6.61524315488,
,6.44980370163,9.59610027855,13.0079792169,3.96455964031,
,6.23441396509,6.10548442576,22.0787746171,2.88935870331,
,7.24637681159,8.13335480754,12.4860894725,5.10440835267,
,8.92420537897,5.00230167255,11.0870610871,7.21767450048,
,11.5942028986,5.02020516009,21.0345524348,7.24405714624,
,7.43639921722,2.40718451995,24.7236180905,4.8075,
,10.2796052632,3.03569559301,24.0720118782,6.19722085

In [242]:
ob

{91: {'Canada': [8.499095840867993,
   11.412535079513564,
   16.842105263157894,
   11.229946524064172,
   7.591623036649215,
   11.556064073226544,
   15.750915750915752,
   17.365269461077844,
   17.43486973947896,
   11.226114649681529,
   6.449803701626473,
   6.234413965087282,
   7.246376811594203,
   8.924205378973106,
   11.594202898550725,
   7.436399217221135,
   10.279605263157896,
   9.837631327602674,
   5.071119356833642,
   10.0,
   17.13426853707415],
  'India': [13.25512169497461,
   4.337175792507205,
   2.2711978465679676,
   3.0210218571627454,
   12.473082656948815,
   3.546193840821224,
   5.487512007684918,
   7.034412955465587,
   7.316053511705686,
   10.261353898886032,
   9.596100278551532,
   6.105484425759602,
   8.133354807537446,
   5.002301672548719,
   5.020205160087038,
   2.4071845199518562,
   3.035695593007432,
   3.705203482486333,
   10.683064157652986,
   5.850991114149009,
   4.265402843601896],
  'United Kingdom': [21.29004140335585,
   31.009

In [ ]:
classes = {91: 'health', 92: 'safety_security', 93 : 'environment',
                      94 : 'social_relations', 95 : 'meaning_in_life', 96 : 'achievement',
                      97 : 'economics', 98 : 'politics', 99 : 'not_applicable', 0 : 'skip' }

In [409]:
a = df[df.created_at > pd.to_datetime("2018-05-25 00:00:00")][df.created_at < pd.to_datetime("2018-05-26 00:00:00")]\
[df.worry == 97][df.country == "United Kingdom"]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [410]:
pd.DataFrame({'counts' : a.groupby( [ "url"] ).size()}).reset_index().sort_values("counts",ascending=False).values[:20]

array([[u'https://www.theguardian.com/business/2018/may/25/uk-economy-posts-worst-quarterly-gdp-figures-for-five-years',
        172],
       [u'https://www.theguardian.com/business/2018/may/25/uk-economy-posts-worst-quarterly-gdp-figures-for-five-years?CMP=share_btn_tw',
        84],
       [u'https://www.theguardian.com/business/2018/may/25/uk-economy-posts-worst-quarterly-gdp-figures-for-five-years?CMP=Share_iOSApp_Other',
        81],
       [u'https://www.theguardian.com/business/2018/may/25/uk-economy-posts-worst-quarterly-gdp-figures-for-five-years?CMP=Share_AndroidApp_Tweet',
        36],
       [u'https://amp.theguardian.com/business/live/2018/may/25/uk-gdp-growth-figures-markets-ftse-sterling-business-live?__twitter_impression=true',
        31],
       [u'https://www.theguardian.com/business/live/2018/may/25/uk-gdp-growth-figures-markets-ftse-sterling-business-live',
        29],
       [u'https://www.theguardian.com/business/live/2018/may/25/uk-gdp-growth-figures-markets-ft

In [338]:
a.shape

(2319, 8)

In [337]:
df[df.created_at > pd.to_datetime("2018-05-14 00:00:00")][df.created_at < pd.to_datetime("2018-05-15 00:00:00")]\
[df.country == "United Kingdom"].shape

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(4925, 8)

In [414]:
df2.to_csv("../../data/all_data.csv", encoding="latin-1")